Source: Example 2 https://realpython.com/pandas-groupby/

The air quality dataset contains hourly readings from a gas sensor device in Italy. Missing values are denoted with -200 in the CSV file. You can use read_csv() to combine two columns into a timestamp while using a subset of the other columns:

In [1]:
import pandas as pd

In [19]:
df_tmp = pd.read_csv('airqual.csv')
df_tmp.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/04,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/04,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/04,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/04,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/04,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888


In [22]:
#df_tmp.isnan.sum()

In [5]:
df = pd.read_csv("airqual.csv",
    parse_dates=[["Date", "Time"]],
    na_values=[-200],
    usecols=["Date", "Time", "CO(GT)", "T", "RH", "AH"]
).rename(
    columns={
        "CO(GT)": "co",
        "Date_Time": "tstamp",
        "T": "temp_c",
        "RH": "rel_hum",
        "AH": "abs_hum",
    }
).set_index("tstamp")

This produces a DataFrame with a DatetimeIndex and four float columns:

In [6]:
df.head()

,co,temp_c,rel_hum,abs_hum
tstamp,,,,
2004-03-10 18:00:00,2.6,13.6,48.9,0.7578
2004-03-10 19:00:00,2.0,13.3,47.7,0.7255
2004-03-10 20:00:00,2.2,11.9,54.0,0.7502
2004-03-10 21:00:00,2.2,11.0,60.0,0.7867
2004-03-10 22:00:00,1.6,11.2,59.6,0.7888


Here, co is that hour’s average carbon monoxide reading, while temp_c, rel_hum, and abs_hum are the average temperature in Celsius, relative humidity, and absolute humidity over that hour, respectively. The observations run from March 2004 through April 2005:

In [8]:
df.index.max()

Timestamp('2005-04-04 14:00:00')

In [9]:
df.index.min()

Timestamp('2004-03-10 18:00:00')

## Grouping on Derived Arrays ##

Earlier you saw that the first parameter to .groupby() can accept several different arguments:

    A column or list of columns
    A dict or Pandas Series
    A NumPy array or Pandas Index, or an array-like iterable of these

You can take advantage of the last option in order to group by the day of the week. You can use the index’s .day_name() to produce a Pandas Index of strings. Here are the first ten observations:

In [14]:
df.index

DatetimeIndex(['2004-03-10 18:00:00', '2004-03-10 19:00:00',
               '2004-03-10 20:00:00', '2004-03-10 21:00:00',
               '2004-03-10 22:00:00', '2004-03-10 23:00:00',
               '2004-03-11 00:00:00', '2004-03-11 01:00:00',
               '2004-03-11 02:00:00', '2004-03-11 03:00:00',
               ...
               '2005-04-04 05:00:00', '2005-04-04 06:00:00',
               '2005-04-04 07:00:00', '2005-04-04 08:00:00',
               '2005-04-04 09:00:00', '2005-04-04 10:00:00',
               '2005-04-04 11:00:00', '2005-04-04 12:00:00',
               '2005-04-04 13:00:00', '2005-04-04 14:00:00'],
              dtype='datetime64[ns]', name='tstamp', length=9357, freq=None)

In [15]:
day_names = df.index.day_name()
type(day_names)

pandas.core.indexes.base.Index

In [16]:
day_names[:10]

Index(['Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Thursday', 'Thursday', 'Thursday', 'Thursday'],
      dtype='object', name='tstamp')

You can then take this object and use it as the .groupby() key. In Pandas-speak, day_names is **array-like**. It’s a one-dimensional sequence of labels.

**Note:** For a Pandas **Series**, rather than an Index, you’ll need the **.dt** accessor to get access to methods like **.day_name()**. If ser is your Series, then you’d need **ser.dt.day_name()**.

Now, pass that object to .groupby() to find the average carbon monoxide ()co) reading by day of the week:

In [17]:
df.groupby(day_names)["co"].mean()

tstamp
Friday       2.543041
Monday       2.016741
Saturday     1.861077
Sunday       1.438069
Thursday     2.455505
Tuesday      2.382267
Wednesday    2.400787
Name: co, dtype: float64

The split-apply-combine process behaves largely the same as before, except that the splitting this time is done on an artificially-created column. This column doesn’t exist in the DataFrame itself, but rather is derived from it.

What if you wanted to group not just by day of the week, but by hour of the day? That result should have 7 * 24 = 168 observations. To accomplish that, you can pass a list of array-like objects. In this case, you’ll pass Pandas Int64Index objects:

In [26]:
df.head()

,co,temp_c,rel_hum,abs_hum
tstamp,,,,
2004-03-10 18:00:00,2.6,13.6,48.9,0.7578
2004-03-10 19:00:00,2.0,13.3,47.7,0.7255
2004-03-10 20:00:00,2.2,11.9,54.0,0.7502
2004-03-10 21:00:00,2.2,11.0,60.0,0.7867
2004-03-10 22:00:00,1.6,11.2,59.6,0.7888


In [27]:
hr = df.index.hour
df.groupby([day_names, hr])["co"].mean().rename_axis(["dow", "hr"])

dow        hr
Friday     0     1.936170
           1     1.608511
           2     1.172340
           3     0.887234
           4     0.823333
                   ...   
Wednesday  19    4.146809
           20    3.844681
           21    2.897872
           22    2.102128
           23    1.938298
Name: co, Length: 168, dtype: float64

Here’s one more similar case that uses **.cut()** to bin the temperature values into discrete intervals:

In [33]:
bins = pd.cut(df["temp_c"], bins=3, labels=("cool", "warm", "hot"))
print(bins)

tstamp
2004-03-10 18:00:00    cool
2004-03-10 19:00:00    cool
2004-03-10 20:00:00    cool
2004-03-10 21:00:00    cool
2004-03-10 22:00:00    cool
                       ... 
2005-04-04 10:00:00    warm
2005-04-04 11:00:00    warm
2005-04-04 12:00:00    warm
2005-04-04 13:00:00    warm
2005-04-04 14:00:00    warm
Name: temp_c, Length: 9357, dtype: category
Categories (3, object): [cool < warm < hot]


In [34]:
type(bins)

pandas.core.series.Series

In [35]:
bins.head()

tstamp
2004-03-10 18:00:00    cool
2004-03-10 19:00:00    cool
2004-03-10 20:00:00    cool
2004-03-10 21:00:00    cool
2004-03-10 22:00:00    cool
Name: temp_c, dtype: category
Categories (3, object): [cool < warm < hot]

In [36]:
df[["rel_hum","abs_hum"]].groupby(bins).agg(["mean","median"])

rel_hum          abs_hum        
             mean median      mean  median
temp_c                                    
cool    57.651452   59.2  0.665874  0.6581
warm    49.382716   49.3  1.182894  1.1452
hot     24.994334   24.1  1.292958  1.2742

Whether it’s a Series, NumPy array, or list doesn’t matter. What’s important is that bins still serves as a sequence of labels, one of cool, warm, or hot. If you really wanted to, then you could also use a Categorical array or even a plain-old list:

    Native Python list: df.groupby(bins.tolist())
    Pandas Categorical array: df.groupby(bins.values)

As you can see, .groupby() is smart and can handle a lot of different input types. Any of these would produce the same result because all of them function as a sequence of labels on which to perform the grouping and splitting.

In [37]:
df.groupby(bins.tolist())

In [38]:
df.groupby(bins.values)

**Resampling**

You’ve grouped df by the day of the week with **df.groupby(day_names)["co"].mean()**. Now consider something different. What if you wanted to group by an observation’s year and quarter? Here’s one way to accomplish that:

In [39]:
# See an easier laternative below
df.groupby([df.index.year, df.index.quarter])["co"].agg(["max", "min"]).rename_axis(["year","quarter"])

max  min
year quarter           
2004 1         8.1  0.3
     2         7.3  0.1
     3         7.5  0.1
     4        11.9  0.1
2005 1         8.7  0.1
     2         5.0  0.3

This whole operation can, alternatively, be expressed through **resampling**. One of the uses of resampling is as a time-based groupby. All that you need to do is pass a frequency string, such as "Q" for "quarterly", and Pandas will do the rest:

In [40]:
df.resample("Q")["co"].agg(["max","min"])

,max,min
tstamp,,
2004-03-31,8.1,0.3
2004-06-30,7.3,0.1
2004-09-30,7.5,0.1
2004-12-31,11.9,0.1
2005-03-31,8.7,0.1
2005-06-30,5.0,0.3


Often, when you use .resample() you can express time-based grouping operations in a much more succinct manner. The result may be a tiny bit different than the more verbose .groupby() equivalent, but you’ll often find that **.resample()** gives you exactly what you’re looking for.